In [5]:
import numpy as np
import pandas as pd

In [2]:
!pip install groq


     -------------------------------------- 138.3/138.3 kB 1.4 MB/s eta 0:00:00
     -------------------------------------- 73.5/73.5 kB 506.2 kB/s eta 0:00:00
     -------------------------------------- 71.0/71.0 kB 653.4 kB/s eta 0:00:00
     ---------------------------------------- 78.8/78.8 kB 1.5 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 2.6
    Uninstalling idna-2.6:
      Successfully uninstalled idna-2.6
  Attempting uninstall: h11
    Found existing installation: h11 0.9.0
    Uninstalling h11-0.9.0:
      Successfully uninstalled h11-0.9.0
  Attempting uninstall: httpcore
    Found existing installation: httpcore 0.9.1
    Uninstalling httpcore-0.9.1:
      Successfully uninstalled httpcore-0.9.1
  Attempting uninstall: httpx
    Found existing installation: httpx 0.13.3
    Uninstalling httpx-0.13.3:
      Successfully uninstalled httpx-0.13.3


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
googletrans 4.0.0rc1 requires httpx==0.13.3, but you have httpx 0.28.1 which is incompatible.
conda-repo-cli 1.0.20 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
conda-repo-cli 1.0.20 requires nbformat==5.4.0, but you have nbformat 5.10.2 which is incompatible.
conda-repo-cli 1.0.20 requires PyYAML==6.0, but you have pyyaml 6.0.2 which is incompatible.
conda-repo-cli 1.0.20 requires requests==2.28.1, but you have requests 2.32.3 which is incompatible.


In [6]:
# !pip install groq pandas tqdm  # install if not already installed

import pandas as pd
from groq import Groq
from tqdm import tqdm


In [7]:
df=pd.read_csv("paysim_fraud_scores.csv")
df.head()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,balance_delta_org,balance_delta_dest,...,type_PAYMENT,type_TRANSFER,ml_fraud_score,ml_anomaly_flag,rule_score,rule_reasons,rule_score_norm,llm_score,total_fraud_score,flagged_fraud
0,278,330218.42,20866.00,351084.42,452419.57,122201.15,0,0,-330218.42,-330218.42,...,False,False,0.214322,0,40,Balance mismatch detected;,0.40,0,0.208593,0
1,15,11647.08,30370.00,18722.92,0.00,0.00,0,0,11647.08,0.00,...,True,False,0.007399,0,0,NaN,0.00,0,0.004439,0
2,10,152264.21,106589.00,258853.21,201303.01,49038.80,0,0,-152264.21,-152264.21,...,False,False,0.124148,0,40,Balance mismatch detected;,0.40,0,0.154489,0
3,403,1551760.63,0.00,0.00,3198359.45,4750120.08,0,0,0.00,1551760.63,...,False,True,0.430251,0,55,Balance mismatch detected; Transaction type is...,0.55,0,0.368150,0
4,206,78172.30,2921331.58,2999503.88,415821.90,337649.60,0,0,-78172.30,-78172.30,...,False,False,0.226824,0,40,Balance mismatch detected;,0.40,0,0.216094,0


In [ ]:
# Replace with your free-tier API key
client = Groq(api_key="**************************")


In [9]:
df[["amount", "ml_fraud_score", "rule_score", "total_fraud_score", "flagged_fraud"]].head()

,amount,ml_fraud_score,rule_score,total_fraud_score,flagged_fraud
0,330218.42,0.214322,40,0.208593,0
1,11647.08,0.007399,0,0.004439,0
2,152264.21,0.124148,40,0.154489,0
3,1551760.63,0.430251,55,0.368150,0
4,78172.30,0.226824,40,0.216094,0


In [10]:
df['flagged_fraud'].value_counts()

flagged_fraud
0    149927
1        73
Name: count, dtype: int64

In [34]:
def investigate_transaction(row):
    # Explicit balance changes
    balance_org_change = row.newbalanceOrig - row.oldbalanceOrg
    balance_dest_change = row.newbalanceDest - row.oldbalanceDest

    prompt = f"""
You are a senior financial fraud investigator.
Write for a fraud analyst who has limited time and needs clarity.

Transaction details:
- Amount: {row.amount}
- Transaction Type: {"TRANSFER" if row.type_TRANSFER else "CASH_OUT" if row.type_CASH_OUT else "OTHER"}

Account balances:
- Sender balance before: {row.oldbalanceOrg}
- Sender balance after: {row.newbalanceOrig}
- Receiver balance before: {row.oldbalanceDest}
- Receiver balance after: {row.newbalanceDest}

Balance changes:
- Sender balance change: {balance_org_change} (negative means loss)
- Receiver balance change: {balance_dest_change} (positive means gain)

Model signals:
- ML Fraud Score (0–1): {row.ml_fraud_score}
- Rule Score (0–100): {row.rule_score}
- Total Fraud Score (0–1): {row.total_fraud_score}

Instructions:
1. Assign a Risk Level using this rule:
   - Low: < 0.5
   - Medium: 0.5–0.7
   - High: > 0.7

2. Respond in the EXACT format below. Do not add extra text.

Risk Level: <Low | Medium | High>

Key Risk:
<1–2 concise sentences explaining the strongest reasons this transaction is risky or not risky.>

Recommended Action:
<1–2 concise sentences describing what the fraud team should do next.>

Rules:
- Be factual and professional.
- Do NOT use bullet points.
- Do NOT speculate beyond the provided data.
"""

    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2
    )

    return response.choices[0].message.content



In [35]:
# Analyze only flagged transactions first
flagged_df = df[df["flagged_fraud"] == 1].copy()

# Create a new column for LLM explanation
llm_explanations = []

for idx, row in tqdm(flagged_df.iterrows(), total=flagged_df.shape[0]):
    explanation = investigate_transaction(row)
    llm_explanations.append(explanation)

# Save explanations
flagged_df["llm_explanation"] = llm_explanations


100%|██████████| 73/73 [04:20<00:00,  3.56s/it]


In [36]:
flagged_df['llm_explanation'].head()

1987     Risk Level: High\n\nKey Risk: The high ML Frau...
2530     Risk Level: High\n\nKey Risk: The transaction'...
3069     Risk Level: High\n\nKey Risk: The high Total F...
3542     Risk Level: High\n\nKey Risk: The transaction'...
11754    Risk Level: High\n\nKey Risk: The transaction ...
Name: llm_explanation, dtype: object

In [37]:
df["llm_explanation"] = "Not investigated (low risk)"


In [38]:
df.loc[flagged_df.index, "llm_explanation"] = flagged_df["llm_explanation"]


In [39]:
for i, text in enumerate(flagged_df["llm_explanation"].head(5)):
    print(f"\n--- LLM Explanation #{i+1} ---")
    print(text)



--- LLM Explanation #1 ---
Risk Level: High

Key Risk: The high ML Fraud Score of 0.9551067927668976 and Total Fraud Score of 0.7030640756601385 indicate a strong likelihood of fraudulent activity. The significant gain of 758074.45 in the receiver's account and the corresponding loss in the sender's account further support this risk assessment.

Recommended Action: The fraud team should investigate the transaction further to determine the legitimacy of the transfer and potential motives behind it.

--- LLM Explanation #2 ---
Risk Level: High

Key Risk: The transaction's high ML Fraud Score (0.945073749470679) and Total Fraud Score (0.7270442496824074) indicate a high likelihood of fraud. The significant transfer amount of 10001912.39 also raises suspicions.

Recommended Action: The fraud team should thoroughly investigate the transaction, verify the sender and receiver accounts, and review any related transactions to determine the authenticity of the transfer.

--- LLM Explanation #3 

In [40]:
output_path=r"C:\Users\MAITHILI\Gen-AI_FraudGuard\paysim_processed_with_scores.csv"

df.to_csv(output_path, index=False)

print("Saved processed dataset to:", output_path)
print("Final shape:", df.shape)

Saved processed dataset to: C:\Users\MAITHILI\Gen-AI_FraudGuard\paysim_processed_with_scores.csv
Final shape: (150000, 24)
